In [ ]:
from thesis_initialise import *

In [ ]:
from abc import ABC as _ABC, ABCMeta as _ABCMeta
import inspect as _inspect
# import weakref as _weakref

from everest.cascade.signature import Signature as _Signature, Inputs as _Inputs
from everest import (
    reseed as _reseed,
    wordhash as _wordhash,
    )
# from everest.h5anchor import disk as _disk

class Schema(_ABCMeta):
    userdefined = False
    schemas = dict()
    inputs = None
    hashID = None
    def __new__(meta, name, bases, dic):
        schema = super().__new__(meta, name, bases, dic)
        hashID = schema.__name__
        if schema.userdefined:
#             schema.script = script = _disk.ToOpen(inspect.getfile(schema))()
            schema.script = script = str(_reseed.digits(12))
            hashID += _wordhash.get_random_proper(2, seed = script)
        schema.hashID = hashID
        if not hashID in (pre := meta.schemas):
            pre[hashID] = schema
            schema.inputs = _Signature(schema.__init__, skip = 1)
            schema.hashID = hashID
        return pre[hashID]
    def __init__(schema, *args, **kwargs):
        schema.cases = dict()
    def _case_new__(schema, *args, **kwargs):
        inputs = schema.inputs.bind(*args, **kwargs)
        if inputs.partial:
            raise TypeError("Partial cases are forbidden.")
        inhash = inputs.hashID
        if inhash in (cases := schema.cases):
            case = cases[inhash]
        else:
            case = cases[inhash] = super().__new__(
                Case,
                f"{schema.__name__}:{inhash}",
                (schema,),
                {**schema.__dict__, **dict(
                    inputs = inputs,
                    hashID = schema.hashID + ':' + inhash,
                    schema = schema,
                    )},
                )
        return case
    def __getitem__(schema, inputs):
        assert isinstance(inputs, _Inputs)
        return schema._case_new__(*inputs.args, **inputs.kwargs)
    def __call__(schema, *args, **kwargs):
        if not isinstance(schema, Case):
            case = schema._case_new__(*args, **kwargs)
            return case(*args, **kwargs)
        return super().__call__(*args, **kwargs)
    def __repr__(schema):
        return f'Everest{type(schema).__name__}({schema.hashID})'

class Case(Schema):
    def __call__(case, *args, **kwargs):
        if hasattr(case, 'process_inputs'):
            args, kwargs = case.process_inputs(args, kwargs)
        inputs = case.inputs.bind(*args, **kwargs)
        inst = super().__call__(*inputs.args, **inputs.kwargs)
        inst.hashID = case.hashID + ';' + str(_reseed.digits(12))
        inst.case = case
        return inst

class Base(_ABC, metaclass = Schema):
    def __init__(self):
        ...
    def __repr__(self):
        return f'EverestInstance({self.hashID})'

class MySchema(Base):
    def __init__(self, a, b,
            c,
            d = 4, # another comment
            /,
            e: int = 5,
            # stuff
                f = 6,
                g = 7,
            h = 8,
            *args,
            # morestuff
                i,
                j = 10,
                # k
                    k0 = 11,
                    k1 = 110,
                    k2 = 1100,
                l = 12,
            m = 13,
            # bonusstuff
                n = 14,
            # morebonusstuff
                o = 15,
            # _ignore
                fee = 'fee', fie = 'fie', foe = 'foe',
                fum = 'fum',
                # subignore
                    boo = 'boo',
            p = 16,
            **kwargs,
            ):
        print(a, b, c, d, e, f, g, h, args, i, j, k0, k1, k2, l, m, n, o, p, kwargs)

# class Funcy(_ABC, metaclass = Schema):
#     def __init__(self,
#             *args,
#             **kwargs
#             ):


# class Ur(Funcy):
#     ...

# class Base(Funcy):
#     ...

# class Derived(Funcy):
#     ...

In [ ]:
from everest.cascade.signature import Inputs

In [ ]:
try:
    mycase = MySchema[Inputs(1, 2, i = 'wooh')]
    raise Exception
except TypeError as exc:
    print(exc)
mycase = MySchema[Inputs(1, 2, 3, i = 'wooh')]

In [ ]:
mycase(hello = 'yeah')